# Project 3 Part 2 API

<mark> ***Use an API to extract box office revenue and profit data to add to your IMDB data and perform exploratory data analysis.***

### Task

Your stakeholder wants you to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

Note: this process can take a long time and may need to run overnight.
Specifications - Financial Data
Your stakeholder would like you to extract and save the results for movies that meet all of the criteria established in part 1 of the project (You should already have a filtered dataframe saved from part one as a csv.gz file)

* [x] As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001

* [x] Each year should be saved as a separate .csv.gz file

Hint: Use the two custom functions from the lessons (Intro to TMDB API, and Efficient TMDB API Calls). Be sure to define these functions prior to calling them in your code!

One function will add the certification (MPGG Rating) to movie.info
The other function will help you append/extend a JSON file with Python
Confirm Your API Function works.

* [x] In order to ensure your function for extracting movie data from TMDB is working, test your function on these 2 movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook"). Make sure that your function runs without error and that it returns the correct movie's data for both test ids.

Once you have retrieved and saved the final results to 2 separate .csv.gz files, move on to a new Exploratory Data Analysis notebook to explore the following questions.

### Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb
import json
import time
import os

### Loading in csvs

In [2]:
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head(2)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama


In [3]:
# changing year to int because the float may be causing an error in later code
basics['startYear'] = basics['startYear'].astype(int)

In [4]:
# confirming
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


## Setting up the API

In [5]:
# loading api-key
with open('/Users/cameron/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
# confirming
login.keys()

dict_keys(['api-key'])

In [6]:
# setting key in tmdb module
tmdb.API_KEY =  login['api-key']

### Setting up folder

In [7]:
folder = "Data/"
os.makedirs(folder, exist_ok=True)
os.listdir(folder);

## Setting up functions

In [8]:
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    info = movie.info()
    releases = movie.releases()

    for c in releases['countries']:
        if c['iso_3166_1' ] == 'US':
           info['certification'] = c['certification']
    
    return info

In [9]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename, 'r+') as file:
        file_data = json.load(file)
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        file.seek(0)
        json.dump(file_data, file)

### Testing Functions

In order to ensure your function for extracting movie data from TMDB is working, test your function on these 2 movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook"). Make sure that your function runs without error and that it returns the correct movie's data for both test ids.

In [10]:
# using function
avengers = get_movie_with_rating('tt0848228')
notebook = get_movie_with_rating('tt0332280')

In [11]:
# confirming results
for i in [avengers, notebook]:
    print(i['title'], i['release_date'], i['certification'])

The Avengers 2012-04-25 PG-13
The Notebook 2004-06-25 PG-13


<mark><u>**Comment:**</u>

<font color='dodgerblue' size=4><i>
Looks like everything worked here
</i></font>

### Loop to Gather Data

In [12]:
# years to get from api call
years_to_get = [2000, 2001]

In [13]:
# list to catch errors
errors = []

In [14]:
for year in tqdm_notebook(years_to_get, desc='YEARS', position=0):
    json_file = f'{folder}tmdb_api_results_{year}.json'
    if os.path.isfile(json_file) == False:
        with open(json_file, 'w') as f:
            json.dump([{'imdb_id':0}], f)

    df = basics.loc[basics['startYear'] == year].copy()
    movie_ids = df['tconst'].copy()
    previous_df = pd.read_json(json_file)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {year}',
                                  position=1,
                                  leave=True):
        try:
            temp = get_movie_with_rating(movie_id)  
            write_json(temp, json_file)
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
        
    final_year_df = pd.read_json(json_file)
    final_year_df.to_csv(f"{folder}final_tmdb_data_{year}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/210 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/243 [00:00<?, ?it/s]

### Checking Results

In [16]:
# checking the number of errors
print(f"- Total errors: {len(errors)}")

# demonstrating that the errors are related to imdb ids not found in the tmdb
count = 0
not_404 = []
for i in errors:
    if '404 Client Error' in str(i[1]):
        count += 1
    else:
        not_404.append(i)
print(f'- 404 Client Errors: {count}')

# printing any errors that are not 404 if they exist
if not_404: 
    print(not_404)

- Total errors: 453
- 404 Client Errors: 453


In [17]:
# taking a look at errors
errors[:3]

[['tt0115937',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0115937?api_key=4d4c9815bcf18b420f748fabcf653225')],
 ['tt0116628',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0116628?api_key=4d4c9815bcf18b420f748fabcf653225')],
 ['tt0118710',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0118710?api_key=4d4c9815bcf18b420f748fabcf653225')]]

In [18]:
# confirming data
df_2000 = pd.read_json('Data/tmdb_api_results_2000.json')
df_2001 = pd.read_json('Data/tmdb_api_results_2001.json')

In [19]:
# confirming
display(len(df_2000), len(df_2001))

1229

1314

In [20]:
# confirming columns
df_2000.columns

Index(['imdb_id', 'adult', 'backdrop_path', 'belongs_to_collection', 'budget',
       'genres', 'homepage', 'id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'certification'],
      dtype='object')

In [23]:
# inspecting movies missing rating
df_2000[df_2000['certification'].isna()].sample(5).T

,627,858,332,949,495
imdb_id,tt0228180,tt0252335,tt0200192,tt0264079,tt0215841
adult,0.0,0.0,0.0,0.0,0.0
backdrop_path,None,/5iNrEilm4jG4hUklek81e5R8AGN.jpg,/n74r1dfTqnArIGC8VNU8k0gHf04.jpg,/e6BaxmNg805FIIBP4eE0CVU3cRy.jpg,/zKfKwKwQmU4VsQsv3afeNMN6jNS.jpg
belongs_to_collection,None,None,None,"{'id': 429231, 'name': 'Taina Collection', 'po...",None
budget,0.0,0.0,0.0,0.0,0.0
genres,"[{'id': 53, 'name': 'Thriller'}]","[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 10751, 'name': 'Family'}, {'id': 14, '...","[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name..."
homepage,,,,,
id,81277.0,140101.0,22705.0,294161.0,58169.0
original_language,en,fr,it,pt,fr
original_title,Dangerous Attraction,Combat d'amour en songe,Tra(sgre)dire,Tainá: Uma Aventura na Amazônia,Hijack Stories


<mark><u>**Comment:**</u>

<font color='dodgerblue' size=4><i>
Looks like everything worked here.  We will be taking a closer look at the data in a separate notebook.
</i></font>